In [23]:
from src.densite_function import *
from src.gaussian_simulation import *
from src.estimators import *
from src.vraisemblance import *
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import geom

In [4]:
np.random.seed(10)

x=np.random.uniform(size=20, low=-10, high=10)
theta=np.random.uniform(size=20, low=-10, high=10)

In [18]:
A=np.matrix(np.identity(20))*0.5
b=theta/2

In [1]:
def estimateur_SS_logvraisemblance(x, theta, A, b, r):
    K=np.random.geometric(p=0.6, size=1)[0]

    z_O=np.array([])
    z_E=np.array([])

    array_w=np.array([])
    array_w_O=np.array([])
    array_w_E=np.array([])

    i=0
    while i<2**K:
        z_i_O=simulate_gaussian_vector(mu=np.matmul(A,x)+b, sigma=(2/3)*np.identity(20))[0]
        z_i_E=simulate_gaussian_vector(mu=np.matmul(A,x)+b, sigma=(2/3)*np.identity(20))[0]

        w_i_E=w(z=z_i_E, x=x, theta=theta, A=A,b=b)
        w_i_O=w(z=z_i_O, x=x, theta=theta, A=A,b=b)

        
        z_O= np.append(z_O, z_i_O)
        z_E= np.append(z_E, z_i_E)

        array_w_O=np.append(array_w_O, w_i_O)
        array_w_E=np.append(array_w_E, w_i_E)
        i+=1

    array_z=np.union1d(z_O, z_E)
    array_w=np.union1d(array_w_O, array_w_E)

    I_0=np.mean(np.log(array_w))

    IWAE_O=np.mean(np.log(array_w_O))
    IWAE_E=np.mean(np.log(array_w_E))
    IWAE_OUE=np.mean(np.log(array_w))

    delta_K=IWAE_OUE-0.5*(IWAE_O+IWAE_E)

    SS=I_0+delta_K/geom.pmf(K, p=r)

    return SS

In [20]:
def estimateur_RR_logvraisemblance(x, theta, A, b, r):
    K=np.random.geometric(p=0.6, size=1)[0]
    k=0
    array_delta_k=np.array([])
    array_proba_K_sup_k=np.array([])

    while k<K:

        z_O=np.array([])
        z_E=np.array([])

        array_w=np.array([])
        array_w_O=np.array([])
        array_w_E=np.array([])

        i=0
        while i<2**K:
            z_i_O=simulate_gaussian_vector(mu=np.matmul(A,x)+b, sigma=(2/3)*np.identity(20))[0]
            z_i_E=simulate_gaussian_vector(mu=np.matmul(A,x)+b, sigma=(2/3)*np.identity(20))[0]

            w_i_E=w(z=z_i_E, x=x, theta=theta, A=A,b=b)
            w_i_O=w(z=z_i_O, x=x, theta=theta, A=A,b=b)

            
            z_O= np.append(z_O, z_i_O)
            z_E= np.append(z_E, z_i_E)

            array_w_O=np.append(array_w_O, w_i_O)
            array_w_E=np.append(array_w_E, w_i_E)
            i+=1

        array_z=np.union1d(z_O, z_E)
        array_w=np.union1d(array_w_O, array_w_E)

        IWAE_O=np.mean(np.log(array_w_O))
        IWAE_E=np.mean(np.log(array_w_E))
        IWAE_OUE=np.mean(np.log(array_w))

        delta_K=IWAE_OUE-0.5*(IWAE_O+IWAE_E)
        proba_k=geom.cdf(K, p=r)

        array_delta_k=np.append(array_delta_k, delta_K)
        array_proba_K_sup_k=np.append(array_proba_K_sup_k, proba_k)

        k+=1
    
    I_0=np.mean(np.log(array_w))
    RR=I_0+np.sum(array_delta_k/array_proba_K_sup_k)

    return RR

In [7]:
SS=estimateur_SS_logvraisemblance(x=x, theta=theta, A=A, b=b, r=0.6)

In [8]:
SS

-427.73037826512933

In [21]:
RR=estimateur_RR_logvraisemblance(x=x, theta=theta, A=A, b=b, r=0.6)

In [22]:
RR

-427.2948490832669

In [9]:
log_vraisemblance(x, theta)

-427.1362945070707